In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA


In [7]:
mydata = pd.read_csv('LAQN.csv')

In [8]:
mydata.head()


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location
0,CD1,Camden - Swiss Cottage,Kerbside,09/07/2016 09:00:00 AM,Day 191 09:00,Low,1,No,No,E05000133,Frognal and Fitzjohns,526633,184391,-0.175284,51.544219,Unknown,09/07/2016 11:05:05 AM,"(51.544219, -0.175284)"
1,BL0,Camden - Bloomsbury,Urban Background,21/09/2017 01:00:00 AM,Day 264 01:00,No data,0,No,No,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,Unknown,21/09/2017 03:05:06 AM,"(51.522287, -0.125847)"
2,CD1,Camden - Swiss Cottage,Kerbside,31/05/2017 12:00:00 PM,Day 151 12:00,Low,1,No,No,E05000133,Frognal and Fitzjohns,526633,184391,-0.175284,51.544219,Unknown,31/05/2017 02:05:08 PM,"(51.544219, -0.175284)"
3,BL0,Camden - Bloomsbury,Urban Background,19/07/2017 07:00:00 PM,Day 200 19:00,Low,1,No,No,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,Unknown,19/07/2017 09:05:05 PM,"(51.522287, -0.125847)"
4,IM1,Camden - Holborn (inmidtown),Kerbside,06/03/2018 12:00:00 AM,Day 065 00:00,No data,0,No,No,E05000138,Holborn and Covent Garden,530527,181502,-0.120194,51.517368,Unknown,06/03/2018 01:08:18 AM,"(51.517368, -0.120194)"


In [9]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83148 entries, 0 to 83147
Data columns (total 18 columns):
Site Code                  83148 non-null object
Site Name                  83148 non-null object
Site Type                  83148 non-null object
Bulletin Date              83148 non-null object
Bulletin Date Formatted    83148 non-null object
NO2 Air Quality Band       83148 non-null object
NO2 Air Quality Index      83148 non-null int64
Latest Capture             83148 non-null object
Latest Week                83148 non-null object
Ward Code                  83132 non-null object
Ward Name                  83132 non-null object
Easting                    83148 non-null int64
Northing                   83148 non-null int64
Longitude                  83148 non-null float64
Latitude                   83148 non-null float64
Spatial Accuracy           83148 non-null object
Last Uploaded              83148 non-null object
Location                   83148 non-null object
dtypes: fl

In [10]:
mydata.describe()


,NO2 Air Quality Index,Easting,Northing,Longitude,Latitude
count,83148.000000,83148.000000,83148.000000,83148.000000,83148.000000
mean,1.119871,529291.392637,182641.790819,-0.137594,51.527896
std,0.740268,1552.864603,1086.945051,0.021990,0.010109
min,0.000000,526630.000000,181501.000000,-0.175284,51.517368
25%,1.000000,529070.500000,181904.250000,-0.140611,51.521057
50%,1.000000,530011.000000,182337.500000,-0.127310,51.524997
75%,2.000000,530224.000000,183098.000000,-0.124434,51.532036
max,6.000000,530529.000000,184392.000000,-0.120194,51.544219


In [24]:
for col in mydata.columns:
    print(col, len(mydata[col].unique()))

Site Code 4
Site Name 5
Site Type 3
Bulletin Date 20468
Bulletin Date Formatted 8745
NO2 Air Quality Band 3
NO2 Air Quality Index 7
Latest Capture 2
Latest Week 2
Ward Code 5
Ward Name 5
Easting 11
Northing 12
Longitude 5
Latitude 5
Last Uploaded 20792
Location 5


In [14]:
mydata.groupby('Latest Capture').count()


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location
Latest Capture,,,,,,,,,,,,,,,,,
No,83144,83144,83144,83144,83144,83144,83144,83144,83128,83128,83144,83144,83144,83144,83144,83144,83144
Yes,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [15]:
mydata['Latest Capture'] = mydata['Latest Capture'].replace('No',0)
mydata['Latest Capture'] = mydata['Latest Capture'].replace('Yes',1)

In [18]:
mydata.groupby('Spatial Accuracy').count()


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Last Uploaded,Location
Spatial Accuracy,,,,,,,,,,,,,,,,,
Unknown,83148,83148,83148,83148,83148,83148,83148,83148,83148,83132,83132,83148,83148,83148,83148,83148,83148


In [19]:
del mydata['Spatial Accuracy']

In [21]:

mydata.groupby('NO2 Air Quality Band').count()

,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Last Uploaded,Location
NO2 Air Quality Band,,,,,,,,,,,,,,,,
Low,66917,66917,66917,66917,66917,66917,66917,66917,66908,66908,66917,66917,66917,66917,66917,66917
Moderate,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
No data,16164,16164,16164,16164,16164,16164,16164,16164,16157,16157,16164,16164,16164,16164,16164,16164


RangeIndex(start=0, stop=83148, step=1)